In [37]:
from os import listdir
from os.path import isfile, join
import pandas as pd

dir_path = "raw/10ul/"
file_list = [join(dir_path, f) for f in listdir(dir_path) if isfile(join(dir_path, f))]

class DataHandler:
    def __init__(self):
        pass     

    def import_raw(self, path):
        full_data = pd.read_csv(path)    
        raw_data = pd.DataFrame()
        list_of_features = ['Epoch Time', 'Z1 BlkTemp', 'Z1 HeatSink', 'Z1 Imea']
        raw_data = pd.concat([raw_data, full_data[list_of_features]], axis=1)    
        return raw_data
    
    def replace_with_period(self, raw_data):
        # Calculate and use delta_t instead of epoch time
        next_epoch = raw_data['Epoch Time'][1::]
        next_epoch.index -= 1
        raw_data['Epoch Time'] = next_epoch - raw_data['Epoch Time']
        raw_data.rename(columns={'Epoch Time': 'Period'})
        return raw_data

    def process_data(raw_data):
        raw_data = self.replace_with_period(raw_data)
        raw_data = self.add_block_rate(raw_data)
        return features, result    